In [200]:
import os

In [201]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [202]:
os.chdir("../")

In [203]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [204]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [205]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    raw_data_dir: Path
    db_file: Path
    customers: str
    products: str
    customers_csv: Path
    products_csv: Path
    

In [206]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [207]:
# create configuration manager 
class ConfigurationManager: 
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH, 
        #schema_filepath = SCHEMA_FILE_PATH,
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        db  = self.config.database
        
        
        create_directories([config.root_dir, config.raw_data_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            raw_data_dir=config.raw_data_dir,
            db_file = db.db_file,
            customers = db.tables.customers,
            products= db.tables.products,
            customers_csv = config.customers_csv,
            products_csv = config.products_csv
            
        )
        
        return data_ingestion_config

In [208]:
import pandas as pd
import sqlite3
from pathlib import Path
from BankProducts.utils.common import read_yaml, create_directories, save_csv_file
from BankProducts import logger


In [209]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def extract_and_save_data(self):
        logger.info(f"Connecting to database: {self.config.db_file}")
        conn = sqlite3.connect(self.config.db_file)

        try:
            logger.info(f"Reading table: {self.config.customers}")
            customers_df = pd.read_sql_query(f"SELECT * FROM {self.config.bank_customers}", conn)

            logger.info(f"Reading table: {self.config.products_table}")
            products_df = pd.read_sql_query(f"SELECT * FROM {self.config.products_catalog}", conn)
        finally:
            conn.close()
            logger.info("Database connection closed.")
            
        # Create directories
        #create_directories([self.config.raw_data_dir])
        create_directories([os.path.dirname(self.config.db_file)])


        # Save files
        customers_df.to_csv(self.config.customers_csv, index=False)
        products_df.to_csv(self.config.products_csv, index=False)
        logging.info(f"Saved customers to: {self.config.customers_csv}")
        logging.info(f"Saved products to: {self.config.products_csv}")


In [210]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_and_save_data()
except Exception as e:
    logger.exception(e)
    raise e


[2025-05-11 23:29:25,133: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-11 23:29:25,135: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-11 23:29:25,137: INFO: common: created directory at: artifacts]
[2025-05-11 23:29:25,139: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-11 23:29:25,141: INFO: common: created directory at: artifacts/data_ingestion/raw_data]
[2025-05-11 23:29:25,142: INFO: 1734609224: Connecting to database: C:/Users/RICH-FILES/Desktop/Datasets/bank_recommender.db]
[2025-05-11 23:29:25,143: INFO: 1734609224: Reading table: customers]
[2025-05-11 23:29:25,144: INFO: 1734609224: Database connection closed.]
[2025-05-11 23:29:25,145: ERROR: 930908772: 'DataIngestionConfig' object has no attribute 'bank_customers']
Traceback (most recent call last):
  File "C:\Users\RICH-FILES\AppData\Local\Temp\ipykernel_1372\930908772.py", line 5, in <module>
    data_ingestion.extract_and_save_data()
  File "C:\User

AttributeError: 'DataIngestionConfig' object has no attribute 'bank_customers'